# TOC
- [](#)
- [](#)
- [](#)
- [](#)
- [](#)
- [](#)
- [](#)

## Demo

In [37]:
class Box<T>(t: T) {
    var value = t
    
    override fun toString(): String {
        return "Box = {value = $value}"
    }
}

In [38]:
val b: Box<Int> = Box<Int>(1)
val box = Box(1)
val box2 = Box("1")

println(b.toString())
println(box.value.javaClass)
println(box2.value.javaClass)

Box = {value = 1}
class java.lang.Integer
class java.lang.String


## Variance

In [39]:
interface Source<out T> {
    fun nextT(): T
}

fun <T> demo(source: Source<T>) {
    val objects: List<T> = List(3) { source.nextT() } // Creating a List from the source
    for (item in objects) {
        println(item)
    }
}

class StringSource(private val data: List<String>) : Source<String> {
    private var index = 0

    override fun nextT(): String {
        if (index < data.size) {
            return data[index++]
        }
        throw NoSuchElementException("No more elements")
    }
}

class NumberSource(private val data: List<Int>) : Source<Int> {
    private var index = 0

    override fun nextT(): Int {
        if (index < data.size) {
            return data[index++]
        }
        throw NoSuchElementException("No more elements")
    }
}

In [40]:
val sourceOfString: Source<String> = StringSource(listOf("apple", "banana", "cherry"))
demo(sourceOfString)

val numberSource: Source<Int> = NumberSource(listOf(1, 2, 3))
demo(numberSource)

apple
banana
cherry
1
2
3


##

##

##

##

##

##